Extract patient_id

In [5]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time

# change to your folder path
os.chdir('/Users/xiao-zy19/Desktop/Johns Hopkins/Biomedical Data Design/EICU Database/eicu-collaborative-research-database-demo-2.0.1')
# os.chdir('/Users/xiao-zy19/Desktop/Johns Hopkins/Biomedical Data Design/EICU/eicu-collaborative-research-database-2.0')

# import diagnosis.csv
df_diagnosis = pd.read_csv('diagnosis.csv')
df_diagnosis.sort_values(by=['patientunitstayid', 'diagnosisoffset'], inplace=True)

# select cardiovascular patients
df_cardiovascular = df_diagnosis[df_diagnosis['diagnosisstring'].str.contains('cardiovascular')]

# get shock patient 
shock_patient = df_cardiovascular[df_cardiovascular['diagnosisstring'].str.contains('shock')]

# get ventricular patient 
ventricular_patient = df_cardiovascular[df_cardiovascular['diagnosisstring'].str.contains('ventricular')]

# get chest pain patient 
chest_pain_patient = df_cardiovascular[df_cardiovascular['diagnosisstring'].str.contains('chest pain')]

# get arrhythmias patient 
arrhythmias_patient = df_cardiovascular[df_cardiovascular['diagnosisstring'].str.contains('arrhythmias')]

# put id together
df_wanted = pd.concat([shock_patient, ventricular_patient, chest_pain_patient, arrhythmias_patient])

# Get the patient ids from df_wanted & sort the patient id
# patient_id_all multiple entry patient's stayid
patient_id_all = df_wanted['patientunitstayid'].unique()
patient_id_all.sort()
print(patient_id_all)

# import patient.csv
df_patient = pd.read_csv('patient.csv')
df_patient.sort_values(by=['patientunitstayid'], inplace=True)
df_patient_buf = df_patient[df_patient['patientunitstayid'].isin(patient_id_all)]
df_1time_patient = df_patient_buf[df_patient_buf['unitvisitnumber']==1]
# print(df_1time_patient)

# select the patient id from df_1time_patient
patient_id = df_1time_patient['patientunitstayid'].unique()
print(f'Total number of patients: {len(patient_id)}')

[ 143870  151179  151900 ... 3351297 3352230 3352231]
Total number of patients: 915


In [6]:
print(patient_id)
type(patient_id)

[ 143870  151179  151900  152954  153972  155961  156308  157427  162842
  165173  165929  166853  172678  175243  176050  184757  186069  186393
  187150  190446  193665  197620  197921  202294  202707  203101  204602
  204935  205061  205928  208726  209139  220469  222133  222408  223303
  223787  224606  225206  229383  230150  230427  234941  242895  243097
  243999  244477  249328  250073  250719  250724  257287  257802  258915
  260468  260647  260860  261520  263850  264276  264423  264901  267196
  275329  276594  276815  280350  281132  282439  284265  284517  288133
  289502  289640  291628  292154  294032  295916  299914  300007  301185
  301780  302668  304256  304631  306247  310446  310783  311043  312258
  312665  313055  313736  326308  327581  329635  332253  333689  335868
  336272  341668  341780  343093  344379  346380  347475  348873  349322
  350259  350263  351717  351720  356684  357166  361404  365045  369237
  369273  370128  371003  375854  376041  378120  3

numpy.ndarray

1. Heart rate

In [7]:
# import vitalPeriodic.csv & nurseCharting.csv
df_vitalPeriodic = pd.read_csv('vitalPeriodic.csv')
df_vitalPeriodic.sort_values(by=['patientunitstayid', 'observationoffset'], inplace=True)
df_nurseCharting = pd.read_csv('nurseCharting.csv')
df_nurseCharting.sort_values(by=['patientunitstayid', 'nursingchartoffset'], inplace=True)


# select the wanted patient
df_vitalPeriodic = df_vitalPeriodic[df_vitalPeriodic['patientunitstayid'].isin(patient_id)]
df_nurseCharting = df_nurseCharting[df_nurseCharting['patientunitstayid'].isin(patient_id)]

# save the wanted file (uncomment the code to save)
# df_vitalPeriodic.to_csv('vitalPeriodic_wanted.csv', index=False)

# print the shape of the wanted file
print(f'vitalperiodic shape: {df_vitalPeriodic.shape}')
print(f'nurseCharting shape: {df_nurseCharting.shape}')

# nursingchartcelltypevallabel Heart Rate
df_nurseCharting = df_nurseCharting[df_nurseCharting['nursingchartcelltypevallabel']=='Heart Rate']
df_nurseCharting = df_nurseCharting.rename(columns={'nursingchartoffset': 'observationoffset', 'nursingchartvalue':'heartrate'})
print(df_nurseCharting.head())

# define heartrate preprocessing function
def normal_heartrate(num):
    """
    Function to normalize heart rate values.

    Parameters:
        num: the originial input value
    Return:
        num: the normalized output value
    """
    # Return null values direcly
    if pd.isna(num):
        return num
    # Remove values out of range
    elif num > 300 or num < 0:
        return np.nan
    # Return normal values directly
    else:
        return num

# extract heart rate from df_vitalPeriodic & df_nurseCharting
HR_v = df_vitalPeriodic[['patientunitstayid', 'observationoffset', 'heartrate']]
HR_n = df_nurseCharting[['patientunitstayid', 'observationoffset', 'heartrate']]
HR = pd.concat([HR_v, HR_n]).astype(float)
HR.sort_values(by=['patientunitstayid', 'observationoffset'], inplace=True)
print(f'First 5 rows of HR: \n{HR.head()} \n')

# exclude abnormal heart rate values
HR.loc[:, 'heartrate'] = HR['heartrate'].apply(normal_heartrate)

# save HR to csv file (uncomment the code to save)
# HR.to_csv('HR.csv', index=False)

value_position_dict = {}
first_occurrences = []
for idx, value in enumerate(HR['patientunitstayid']):
    # if the value is not in the dictionary, add it and create index
    if value not in value_position_dict:
        value_position_dict[value] = idx
        first_occurrences.append(idx)

first_occurrences.append(len(HR))
# create first occurrence index for every patient
HR_index = pd.Series(first_occurrences)
print(f'shape of HR_index: {HR_index.shape}')
print(f'First 5 rows of HR_index: \n{HR_index.head()}')

# double check the index is correct
# print(HR.iloc[HR_index].head())
# print(HR.iloc[[156, 157, 158, 159]])
# print(HR.iloc[[1015, 1016, 1017, 1018]])

vitalperiodic shape: (747487, 19)
nurseCharting shape: (603027, 8)
     nursingchartid  patientunitstayid  observationoffset  \
130       200873835             143870               -424   
155        91951972             143870                -67   
346       146117804             143870                -57   
344       146377143             143870                -17   
169       182492893             143870                 -2   

     nursingchartentryoffset nursingchartcelltypecat  \
130                     -424             Vital Signs   
155                      -67             Vital Signs   
346                      -57             Vital Signs   
344                      -17             Vital Signs   
169                       -2             Vital Signs   

    nursingchartcelltypevallabel nursingchartcelltypevalname heartrate  
130                   Heart Rate                  Heart Rate        53  
155                   Heart Rate                  Heart Rate        55  
346       

In [8]:
# 第一次进入ICU的心血管病疾病患者ID集合
first_time_icu_patients = set(patient_id)
print(f'Number of first time ICU patients: {len(first_time_icu_patients)}')

# 有heartrate数据的患者ID集合
patients_with_heartrate = set(HR['patientunitstayid'].unique())
print(f'Number of patients with heart rate data: {len(patients_with_heartrate)}')

# 没有heartrate数据的患者ID集合
# patients_without_heartrate = first_time_icu_patients - patients_with_heartrate
# print(f'Number of patients without heart rate data: {len(patients_without_heartrate)}')

Number of first time ICU patients: 915
Number of patients with heart rate data: 913


2. Temperature

In [9]:
# import vitalPeriodic.csv & nurseCharting.csv
df_vitalPeriodic = pd.read_csv('vitalPeriodic.csv')
df_vitalPeriodic.sort_values(by=['patientunitstayid', 'observationoffset'], inplace=True)
df_nurseCharting = pd.read_csv('nurseCharting.csv')
df_nurseCharting.sort_values(by=['patientunitstayid', 'nursingchartoffset'], inplace=True)


# select the wanted patient
df_vitalPeriodic = df_vitalPeriodic[df_vitalPeriodic['patientunitstayid'].isin(patient_id)]
df_nurseCharting = df_nurseCharting[df_nurseCharting['patientunitstayid'].isin(patient_id)]

# save the wanted file (uncomment the code to save)
# df_vitalPeriodic.to_csv('vitalPeriodic_wanted.csv', index=False)

# print the shape of the wanted file
print(f'vitalperiodic shape: {df_vitalPeriodic.shape}')
print(f'nurseCharting shape: {df_nurseCharting.shape}')
# define temperature transformation function
def normal_temperature(num):
    """
    Function to normalize temperature values.

    Parameters:
        num: the originial input value
    Return:
        num: the normalized output value
    """
    # Return null values direcly
    if pd.isna(num):
        return num
    # Convert Fahrenheit to Celsius
    # And apply the function again
    elif num > 50:
        return normal_temperature((num - 32) * 5 / 9)
    # Remove values out of range
    elif num < 15 or num > 45:
        return np.nan
    # Return normal values directly
    else:
        return num
# nursingchartcelltypevallabel Temperature
df_nurseCharting = df_nurseCharting[df_nurseCharting['nursingchartcelltypevallabel']=='Temperature']
df_nurseCharting = df_nurseCharting.rename(columns={'nursingchartoffset': 'observationoffset', 'nursingchartvalue':'temperature'})
print(df_nurseCharting.head())

# extract temperature from df_vitalPeriodic & df_nurseCharting
Temp_v = df_vitalPeriodic[['patientunitstayid', 'observationoffset', 'temperature']]
Temp_n = df_nurseCharting[['patientunitstayid', 'observationoffset', 'temperature']]

# delete the rows with string values
Temp_n = Temp_n[Temp_n['temperature'].apply(lambda x: str(x).replace('.', '', 1).isdigit())]
print(f'First 5 rows of Temp_n: \n{Temp_n.head()} \n')

Temp = pd.concat([Temp_v, Temp_n]).astype(float)
Temp.sort_values(by=['patientunitstayid', 'observationoffset'], inplace=True)
print(f'First 5 rows of Temp: \n{Temp.head()} \n')

# exclude abnormal heart rate values
Temp.loc[:, 'temperature'] = Temp['temperature'].apply(normal_temperature)

# save Temp to csv file (uncomment the code to save)
# Temp.to_csv('Temp.csv', index=False)

value_position_dict = {}
first_occurrences = []
for idx, value in enumerate(Temp['patientunitstayid']):
    # if the value is not in the dictionary, add it and create index
    if value not in value_position_dict:
        value_position_dict[value] = idx
        first_occurrences.append(idx)

first_occurrences.append(len(Temp))
# create first occurrence index for every patient
Temp_index = pd.Series(first_occurrences)
print(f'First 5 rows of Temp_index: \n{Temp_index.head()}')

vitalperiodic shape: (747487, 19)
nurseCharting shape: (603027, 8)
     nursingchartid  patientunitstayid  observationoffset  \
58         95478972             143870               -424   
94         95478973             143870               -424   
119        95478971             143870               -424   
316       240209719             143870                -72   
343       240209720             143870                -72   

     nursingchartentryoffset nursingchartcelltypecat  \
58                      -424             Vital Signs   
94                      -424             Vital Signs   
119                     -424             Vital Signs   
316                      -72             Vital Signs   
343                      -72             Vital Signs   

    nursingchartcelltypevallabel nursingchartcelltypevalname      temperature  
58                   Temperature        Temperature Location  TEMPORAL ARTERY  
94                   Temperature             Temperature (F)         

In [10]:
# 第一次进入ICU的心血管疾病患者ID集合
first_time_icu_patients = set(patient_id)
print(f'First time ICU patients: {len(first_time_icu_patients)}')

# 有温度数据的患者ID集合
patients_with_temperature = set(Temp['patientunitstayid'].unique())
print(f'Patients with temperature: {len(patients_with_temperature)}')

# 没有温度数据的患者ID集合
# patients_without_temperature = first_time_icu_patients - patients_with_temperature
# print(f'Patients without temperature: {len(patients_without_temperature)}')
# print(f'Patients without temperature: \n {patients_without_temperature}')

First time ICU patients: 915
Patients with temperature: 915


3. Blood pressure

In [11]:
# import vitalPeriodic.csv & nurseCharting.csv
df_vitalPeriodic = pd.read_csv("vitalPeriodic.csv")
df_vitalPeriodic.sort_values(
    by=["patientunitstayid", "observationoffset"], inplace=True
)
df_nurseCharting = pd.read_csv("nurseCharting.csv")
df_nurseCharting.sort_values(
    by=["patientunitstayid", "nursingchartoffset"], inplace=True
)
df_vitalAPeriodic = pd.read_csv("vitalAperiodic.csv")
df_vitalAPeriodic.sort_values(
    by=["patientunitstayid", "observationoffset"], inplace=True
)


# select the wanted patient
df_vitalPeriodic = df_vitalPeriodic[
    df_vitalPeriodic["patientunitstayid"].isin(patient_id)
]
df_nurseCharting = df_nurseCharting[
    df_nurseCharting["patientunitstayid"].isin(patient_id)
]
df_vitalAPeriodic = df_vitalAPeriodic[
    df_vitalAPeriodic["patientunitstayid"].isin(patient_id)
]

# save the wanted file (uncomment the code to save)
# df_vitalPeriodic.to_csv('vitalPeriodic_wanted.csv', index=False)
# df_nurseCharting.to_csv('nurseCharting_wanted.csv', index=False)
# df_vitalAPeriodic.to_csv('vitalAPeriodic_wanted.csv', index=False)

# print the shape of the wanted file
print(f"vitalperiodic shape: {df_vitalPeriodic.shape}")
print(f"nurseCharting shape: {df_nurseCharting.shape}")
print(f"vitalAPeriodic shape: {df_vitalAPeriodic.shape}")

# nursingchartcelltypevallabel Non-Invasive BP Systolic
df_nurseCharting_noninvasive = df_nurseCharting[
    df_nurseCharting["nursingchartcelltypevalname"] == "Non-Invasive BP Systolic"
]
df_nurseCharting_noninvasive = df_nurseCharting_noninvasive.rename(
    columns={
        "nursingchartoffset": "observationoffset",
        "nursingchartvalue": "Non-Invasive BP Systolic",
    }
)
# print(df_nurseCharting_noninvasive.head())

# nursingchartcelltypevallabel Invasive BP Systolic
df_nurseCharting_invasive = df_nurseCharting[
    df_nurseCharting["nursingchartcelltypevalname"] == "Invasive BP Systolic"
]
df_nurseCharting_invasive = df_nurseCharting_invasive.rename(
    columns={
        "nursingchartoffset": "observationoffset",
        "nursingchartvalue": "Invasive BP Systolic",
    }
)
# print(df_nurseCharting_invasive.head())

# extract systolics from vitalPeriodic, nurseCharting & vitalAPeriodic
systemicsystolic = df_vitalPeriodic[
    ["patientunitstayid", "observationoffset", "systemicsystolic"]
]
non_invasive_BP_Systolic = df_nurseCharting_noninvasive[
    ["patientunitstayid", "observationoffset", "Non-Invasive BP Systolic"]
]
invasive_BP_Systolic = df_nurseCharting_invasive[
    ["patientunitstayid", "observationoffset", "Invasive BP Systolic"]
]
Noninvasivesystolic = df_vitalAPeriodic[
    ["patientunitstayid", "observationoffset", "noninvasivesystolic"]
]


def create_index(df, column_name="patientunitstayid"):
    value_position_dict = {}
    first_occurrences = []

    for idx, value in enumerate(df[column_name]):
        if value not in value_position_dict:
            value_position_dict[value] = idx
            first_occurrences.append(idx)

    first_occurrences.append(len(df))
    return pd.Series(first_occurrences)


# create index for each variable
systemicsystolic_index = create_index(systemicsystolic)
non_invasive_BP_Systolic_index = create_index(non_invasive_BP_Systolic)
invasive_BP_Systolic_index = create_index(invasive_BP_Systolic)
Noninvasivesystolic_index = create_index(Noninvasivesystolic)

vitalperiodic shape: (747487, 19)
nurseCharting shape: (603027, 8)
vitalAPeriodic shape: (129156, 13)


In [12]:
# 第一次进入ICU的心血管疾病患者ID集合
first_time_icu_patients = set(patient_id)
print(f'First time ICU patients: {len(first_time_icu_patients)}')

# 有systemicsystolic noninvasivesystolic invasive BP Systolic non_invasive_BP Systolic的患者ID集合
patients_with_systolic = set(systemicsystolic["patientunitstayid"].unique())
patients_with_noninvasive = set(non_invasive_BP_Systolic["patientunitstayid"].unique())
patients_with_invasive = set(invasive_BP_Systolic["patientunitstayid"].unique())
patients_with_Noninvasive = set(Noninvasivesystolic["patientunitstayid"].unique())

# 求并集
patients_with_bp = patients_with_systolic.union(patients_with_noninvasive, patients_with_invasive, patients_with_Noninvasive)

# 求交集长度
print(f'Patients with all systolic: {len(patients_with_bp)}')

First time ICU patients: 915
Patients with all systolic: 913


4. Glasgow Coma Scale

In [13]:
# import nurseCharting.csv
df_nurseCharting = pd.read_csv('nurseCharting.csv')
df_nurseCharting.sort_values(by=['patientunitstayid', 'nursingchartoffset'], inplace=True)

# select the wanted patient
df_nurseCharting = df_nurseCharting[df_nurseCharting['patientunitstayid'].isin(patient_id)]

# print the shape of the wanted file
print(f'nurseCharting shape: {df_nurseCharting.shape}')

# nursingchartcelltypevallabel Temperature
df_nurseCharting = df_nurseCharting[df_nurseCharting['nursingchartcelltypevallabel']=='Glasgow coma score']
df_nurseCharting = df_nurseCharting.rename(columns={'nursingchartoffset': 'observationoffset', 'nursingchartvalue':'Glasgow score'})
print(df_nurseCharting.head())
print(df_nurseCharting.shape)

# extract glasgow score from df_nurseCharting
Glasgow = df_nurseCharting[['patientunitstayid', 'observationoffset', 'Glasgow score']].copy()
Glasgow.sort_values(by=['patientunitstayid', 'observationoffset'], inplace=True)
print(f'First 5 rows of Temp: \n{Glasgow.head()} \n')

value_position_dict = {}
first_occurrences = []
for idx, value in enumerate(Glasgow['patientunitstayid']):
    # if the value is not in the dictionary, add it and create index
    if value not in value_position_dict:
        value_position_dict[value] = idx
        first_occurrences.append(idx)

first_occurrences.append(len(Glasgow))
# create first occurrence index for every patient
Glasgow_index = pd.Series(first_occurrences)
print(f'First 5 rows of Temp_index: \n{Glasgow_index.head()}')

nurseCharting shape: (603027, 8)
     nursingchartid  patientunitstayid  observationoffset  \
90        265165334             143870                  3   
199       265197766             143870                239   
178       120775807             143870                433   
175        84619753             143870                763   
82        174911019             143870                962   

     nursingchartentryoffset nursingchartcelltypecat  \
90                         3                  Scores   
199                      239                  Scores   
178                      433                  Scores   
175                      763                  Scores   
82                       962                  Scores   

    nursingchartcelltypevallabel nursingchartcelltypevalname Glasgow score  
90            Glasgow coma score                   GCS Total            15  
199           Glasgow coma score                   GCS Total            15  
178           Glasgow coma score

In [14]:
# 第一次进入ICU的心血管疾病患者ID集合
first_time_icu_patients = set(patient_id)
print(f'First time ICU patients: {len(first_time_icu_patients)}')

# 有格拉斯哥昏迷评分数据的患者ID集合
patients_with_glasgow = set(Glasgow['patientunitstayid'].unique())
print(f'Patients with glasgow: {len(patients_with_glasgow)}')

# 没有格拉斯哥昏迷评分数据的患者ID集合
# patients_without_glasgow = first_time_icu_patients - patients_with_glasgow
# print(f'Patients without glasgow: {len(patients_without_glasgow)}')
# print(patients_without_glasgow)


First time ICU patients: 915
Patients with glasgow: 592


5. lab results

In [15]:
# import lab.csv
df_lab = pd.read_csv('lab.csv')
df_lab.sort_values(by=['patientunitstayid', 'labresultoffset'], inplace=True)

# select the wanted patient
df_lab = df_lab[df_lab['patientunitstayid'].isin(patient_id)]

# print the shape of the wanted file
print(f'lab shape: {df_lab.shape}')

def create_index(df):
    """
    Create first occurrence index for every patient
    """
    # create a dictionary to store the first occurrence index
    value_position_dict = {}
    first_occurrences = []
    for idx, value in enumerate(df['patientunitstayid']):
        # if the value is not in the dictionary, add it and create index
        if value not in value_position_dict:
            value_position_dict[value] = idx
            first_occurrences.append(idx)

    first_occurrences.append(len(df))
    # create first occurrence index for every patient
    df_index = pd.Series(first_occurrences)
    return df_index

# lab BUN
df_lab_BUN = df_lab[df_lab['labname']=='BUN']
df_lab_BUN = df_lab_BUN.rename(columns={'labresultoffset': 'observationoffset', 'labresult':'BUN'})

# lab WBC x 1000
df_lab_WBC = df_lab[df_lab['labname']=='WBC x 1000']
df_lab_WBC = df_lab_WBC.rename(columns={'labresultoffset': 'observationoffset', 'labresult':'WBC x 1000'})

# lab bicarbonate
df_lab_bicarbonate = df_lab[df_lab['labname']=='bicarbonate']
df_lab_bicarbonate = df_lab_bicarbonate.rename(columns={'labresultoffset': 'observationoffset', 'labresult':'bicarbonate'})

# lab sodium
df_lab_sodium = df_lab[df_lab['labname']=='sodium']
df_lab_sodium = df_lab_sodium.rename(columns={'labresultoffset': 'observationoffset', 'labresult':'sodium'})

# lab potassium
df_lab_potassium = df_lab[df_lab['labname']=='potassium']
df_lab_potassium = df_lab_potassium.rename(columns={'labresultoffset': 'observationoffset', 'labresult':'potassium'})

# lab total bilirubin
df_lab_bilirubin = df_lab[df_lab['labname']=='total bilirubin']
df_lab_bilirubin = df_lab_bilirubin.rename(columns={'labresultoffset': 'observationoffset', 'labresult':'total bilirubin'})

# extract lab results from lab.csv
BUN = df_lab_BUN[['patientunitstayid', 'observationoffset', 'BUN']].copy()
WBC = df_lab_WBC[['patientunitstayid', 'observationoffset', 'WBC x 1000']].copy()
bicarbonate = df_lab_bicarbonate[['patientunitstayid', 'observationoffset', 'bicarbonate']].copy()
sodium = df_lab_sodium[['patientunitstayid', 'observationoffset', 'sodium']].copy()
potassium = df_lab_potassium[['patientunitstayid', 'observationoffset', 'potassium']].copy()
bilirubin = df_lab_bilirubin[['patientunitstayid', 'observationoffset', 'total bilirubin']].copy()

# sort the lab results by patient id and observationoffset
BUN.sort_values(by=['patientunitstayid', 'observationoffset'], inplace=True)
WBC.sort_values(by=['patientunitstayid', 'observationoffset'], inplace=True)
bicarbonate.sort_values(by=['patientunitstayid', 'observationoffset'], inplace=True)
sodium.sort_values(by=['patientunitstayid', 'observationoffset'], inplace=True)
potassium.sort_values(by=['patientunitstayid', 'observationoffset'], inplace=True)
bilirubin.sort_values(by=['patientunitstayid', 'observationoffset'], inplace=True)

# print the head of the lab results
# print(f'First 5 rows of lab_BUN: \n{BUN.head()} \n')
# print(f'First 5 rows of lab_WBC: \n{WBC.head()} \n')
# print(f'First 5 rows of lab_bicarbonate: \n{bicarbonate.head()} \n')
# print(f'First 5 rows of lab_sodium: \n{sodium.head()} \n')
# print(f'First 5 rows of lab_potassium: \n{potassium.head()} \n')
print(f'First 5 rows of lab_bilirubin: \n{bilirubin.head()} \n')

# create index for each variable
BUN_index = create_index(BUN)
WBC_index = create_index(WBC)
bicarbonate_index = create_index(bicarbonate)
sodium_index = create_index(sodium)
potassium_index = create_index(potassium)
bilirubin_index = create_index(bilirubin)

lab shape: (182390, 10)
First 5 rows of lab_bilirubin: 
        patientunitstayid  observationoffset  total bilirubin
341828             151179               -187              0.4
341827             151179                144              0.2
76036              151900               -265              0.5
23626              152954               -143              0.3
23763              152954                494              0.4 



In [16]:
# 第一次进入ICU患者ID集合
first_time_icu_patients = set(patient_id)
print(f'First time ICU patients: {len(first_time_icu_patients)}')

# 包含BUN WBC bicarbonate sodium potassium bilirubin的患者ID集合
BUN_patients = set(BUN['patientunitstayid'])
WBC_patients = set(WBC['patientunitstayid'])
bicarbonate_patients = set(bicarbonate['patientunitstayid'])
sodium_patients = set(sodium['patientunitstayid'])
potassium_patients = set(potassium['patientunitstayid'])
bilirubin_patients = set(bilirubin['patientunitstayid'])

# 求交集
lab_patients = BUN_patients & WBC_patients & bicarbonate_patients & sodium_patients & potassium_patients & bilirubin_patients
print(f'patients with all lab results: {len(lab_patients)}')

First time ICU patients: 915
patients with all lab results: 664


6. PAO2/FIO2

In [17]:
def create_index(df):
    """
    Create first occurrence index for every patient
    """
    # create a dictionary to store the first occurrence index
    value_position_dict = {}
    first_occurrences = []
    for idx, value in enumerate(df["patientunitstayid"]):
        # if the value is not in the dictionary, add it and create index
        if value not in value_position_dict:
            value_position_dict[value] = idx
            first_occurrences.append(idx)

    first_occurrences.append(len(df))
    # create first occurrence index for every patient
    df_index = pd.Series(first_occurrences)
    return df_index

# def percentage_to_float(percentage):
#     try:
#             # 去掉百分号，将百分数字符串转换为浮点数
#         return float(percentage.rstrip("%"))
#     except ValueError:
#         return None
    
def percentage_to_float(value):
    # Check if the value is a string and contains a percentage sign
    if isinstance(value, str) and "%" in value:
        try:
            # Remove the percentage sign and convert to float
            return float(value.rstrip("%"))
        except ValueError:
            # Return None if conversion fails
            return None
    # If the value is already a number (float or int), return it directly
    elif isinstance(value, (float, int)):
        return value
    # Return None for other types
    else:
        return None
    
file_name1="nurseCharting.csv"
file_name2="lab.csv"
file_name3="respiratoryCharting.csv"
print("Loading pao2/fio2 Data...")
start_time = time.time()
df_nurseCharting = pd.read_csv(file_name1)
df_nurseCharting.sort_values(
    by=["patientunitstayid", "nursingchartoffset"], inplace=True
)
df_lab = pd.read_csv(file_name2)
df_lab.sort_values(by=["patientunitstayid", "labresultoffset"], inplace=True)
df_respiratoryCharting = pd.read_csv(file_name3)
df_respiratoryCharting.sort_values(
     by=["patientunitstayid", "respchartoffset"], inplace=True
)

    # select the wanted patient
df_nurseCharting = df_nurseCharting[
    df_nurseCharting["patientunitstayid"].isin(patient_id)
]
df_lab = df_lab[df_lab["patientunitstayid"].isin(patient_id)]
df_respiratoryCharting = df_respiratoryCharting[
    df_respiratoryCharting["patientunitstayid"].isin(patient_id)
]

df_nurseCharting_SVO = df_nurseCharting[
    df_nurseCharting["nursingchartcelltypevallabel"] == "SVO2"
]
df_nurseCharting_SVO = df_nurseCharting_SVO.rename(
    columns={"nursingchartoffset": "observationoffset", "nursingchartvalue": "SVO2"}
)

    # nursingchartcelltypevallabel: O2 Saturation
df_nurseCharting_O2 = df_nurseCharting[
    df_nurseCharting["nursingchartcelltypevallabel"] == "O2 Saturation"
]
df_nurseCharting_O2 = df_nurseCharting_O2.rename(
    columns={
        "nursingchartoffset": "observationoffset",
        "nursingchartvalue": "O2 Saturation",
    }
)

    # labname: FiO2
df_lab_FiO2 = df_lab[df_lab["labname"] == "FiO2"]
df_lab_FiO2 = df_lab_FiO2.rename(
    columns={"labresultoffset": "observationoffset", "labresult": "FiO2"}
)

    # labname: paO2
df_lab_paO2 = df_lab[df_lab["labname"] == "paO2"]
df_lab_paO2 = df_lab_paO2.rename(
    columns={"labresultoffset": "observationoffset", "labresult": "paO2"}
)

    # respchartvaluelabel: FiO2
df_respiratoryCharting_FiO2 = df_respiratoryCharting[
    df_respiratoryCharting["respchartvaluelabel"] == "FiO2"
]
df_respiratoryCharting_FiO2 = df_respiratoryCharting_FiO2.rename(
    columns={"respchartoffset": "observationoffset","respchartvalue": "FiO2"}
)

    # respchartvaluelabel: FIO2 (%)
df_respiratoryCharting_FIO2_percent = df_respiratoryCharting[
    df_respiratoryCharting["respchartvaluelabel"] == "FIO2 (%)"
]
df_respiratoryCharting_FIO2_percent = df_respiratoryCharting_FIO2_percent.rename(
    columns={"respchartoffset": "observationoffset", "respchartvalue": "FiO2 (%)"}
)

nurse_SVO2 = (
    df_nurseCharting_SVO[["patientunitstayid", "observationoffset", "SVO2"]]
    .copy()
    .astype("float64")
)
nurse_O2 = (
    df_nurseCharting_O2[["patientunitstayid", "observationoffset", "O2 Saturation"]]
    .copy()
    .astype("float64")
)
nurse_SVO2.sort_values(by=["patientunitstayid", "observationoffset"], inplace=True)
nurse_O2.sort_values(by=["patientunitstayid", "observationoffset"], inplace=True)

    # extract data from df_respiratoryCharting and sort by patientunitstayid and observationoffset
lab_FiO2 = (
    df_lab_FiO2[["patientunitstayid", "observationoffset", "FiO2"]]
    .copy()
    .astype("float64")
)
lab_paO2 = (
    df_lab_paO2[["patientunitstayid", "observationoffset", "paO2"]]
    .copy()
    .astype("float64")
)
lab_FiO2.sort_values(by=["patientunitstayid", "observationoffset"], inplace=True)
lab_paO2.sort_values(by=["patientunitstayid", "observationoffset"], inplace=True)

    # extract data from df_respiratoryCharting and sort by patientunitstayid and observationoffset
resp_FiO2 = df_respiratoryCharting_FiO2[
    ["patientunitstayid", "observationoffset", "FiO2"]
].copy()
resp_FiO2_percent = df_respiratoryCharting_FIO2_percent[
    ["patientunitstayid", "observationoffset", "FiO2 (%)"]
].copy()

    # 将'FiO2'列中的百分数转换为浮点数
resp_FiO2["FiO2"] = resp_FiO2["FiO2"].apply(lambda x: percentage_to_float(x))
resp_FiO2_percent["FiO2 (%)"] = resp_FiO2_percent["FiO2 (%)"].apply(
    lambda x: percentage_to_float(x)
)
resp_FiO2.sort_values(by=["patientunitstayid", "observationoffset"], inplace=True)
resp_FiO2_percent.sort_values(
    by=["patientunitstayid", "observationoffset"], inplace=True
)
f_FIO2 = pd.merge(
    resp_FiO2,
    lab_FiO2,
    how="outer",
    on=["patientunitstayid", "observationoffset", "FiO2"],
)
f_FIO2.sort_values(by=["patientunitstayid", "observationoffset"], inplace=True)
fio2_index = create_index(f_FIO2)
pao2_index = create_index(lab_paO2)
end_time = time.time()
print(f"pao2/fio2 Data Loaded. Time: {end_time - start_time:.2f}s")

Loading pao2/fio2 Data...
pao2/fio2 Data Loaded. Time: 1.21s


In [18]:
patients_with_fio2 = set(f_FIO2['patientunitstayid'].unique())
print(f'Patients with fio2: {len(patients_with_fio2)}')
patients_with_pao2 = set(lab_paO2['patientunitstayid'].unique())
print(f'Patients with pao2: {len(patients_with_pao2)}')

Patients with fio2: 529
Patients with pao2: 396


7. urine

In [19]:
from utils import urine

Urine, _ = urine(patient_id)

Loading Urine Data...
Urine Data Loaded. Time: 0.09s


In [20]:
patients_with_urine = set(Urine['patientunitstayid'].unique())
print(f'Patients with urine: {len(patients_with_urine)}')

Patients with urine: 667


Final: 取所有patient的交集

In [21]:
Final_available_patients = patients_with_heartrate & patients_with_temperature & patients_with_bp & patients_with_glasgow & lab_patients & patients_with_fio2 & patients_with_pao2 & patients_with_urine
print(f'Final available patients: {len(Final_available_patients)}')

Final available patients: 152


In [25]:
# save Final_available_patients to csv file
df_final_available_patients = pd.DataFrame({'patientunitstayid': list(Final_available_patients)})
df_final_available_patients.to_csv('Final_available_patients.csv', index=False)

# import Final_available_patients.csv
df_final_available_patients = pd.read_csv('Final_available_patients.csv')
Final_available_patients = df_final_available_patients['patientunitstayid'].unique()
print(f'Final available patients: {len(Final_available_patients)}')
print(Final_available_patients)

Final available patients: 152
[2283009 1090567 3130903 1252888 3133465  384028 3099676 3098657 3116577
 2606115 2748974 3154481 1735220 1774133  343093  447543  202294 2419768
 1113145  963136  859201 1837638  389703  756295 2798666  302668 1334350
  468560 1588817 3172946  264276 1062496 1068646  276594 2114686  304256
  301185 1474690 1131143 3142795 3046540 1056912  851094 1559197  498848
 1588896  913057  968871 3095720 2558122 3157676 2721454  968367 3237558
 1751224  564216 3143360 3154626 2808515 1567943  222408 3150025 1059533
 1571533  242895 1061585  939222 3157219 3136745 3163372  379119  521456
  209139  585971 1072883 3149046 3178242  187150  443664 3159826 3140371
 3149587 1811733  341780  640796  197921 3230503  472359  688938 1363755
  291628 1568559 1060144  490809 2754365  931649  977218 3348292 1775944
  520522 1441610  276815  151900 3239262  250719  250724  284517 3341168
  260468  152954  383358 2634628 2075529  483209 1824650 1837964 1749391
  261520  506770  243